Listázzuk a foglalások számát szobánként, azon belül a foglalás éve szerint csoportosítva!

- Az évnél a Mettől oszlopot használjuk
- Jelenítsük meg a részösszegeket és a végösszeget is úgy, hogy ne jelenjen meg minden Oszlop/NULL kombináció!
- A részösszegeket jelöljük megfelelően, pl. Részösszeg vagy Szoba összesen

In [ ]:
SELECT IIF(GROUPING_ID(SZOBA_ID)=1,'Összesen',cast(SZOBA_ID as varchar(10))),
    case GROUPING_ID(year(METTOL))
        when 0 then cast(year(METTOL) AS varchar(10))
        when 1 then 'Részösszeg'
        when 3 then 'Összesen'
    END,
       count(*)
 FROM szoba sz join Szallashely szh on sz.SZALLAS_FK = szh.SZALLAS_ID
              join Foglalas f on f.SZOBA_FK =sz.SZOBA_ID
GROUP BY rollup (SZOBA_ID,YEAR(METTOL))

In [ ]:
SELECT  SZOBA_FK,
        FOGLALAS_PK,
        LAG(DATEDIFF(DAY,METTOL,MEDDIG)) OVER(PARTITION BY SZOBA_FK ORDER BY METTOL)
FROM Foglalas
WHERE SZOBA_FK IS NOT NULL
ORDER BY SZOBA_FK

In [ ]:
SELECT szh.SZALLAS_NEV, 
        count(f.SZOBA_FK) 
from Vendeg v LEFT JOIN Foglalas f ON v.USERNEV = f.UGYFEL_FK
    LEFT JOIN Szoba sz ON f.SZOBA_FK = sz.SZOBA_ID
    LEFT JOIN Szallashely szh ON sz.SZALLAS_FK = szh.SZALLAS_ID
WHERE SZALLAS_NEV is not null
group by SZALLAS_NEV

Kérdezzük le az átlagosnál kevesebb főre történt foglalások adatait!

a. Egy új oszlopban jelenjen meg az ügyfél neve is!

b. Egy másik új oszlop mutassa meg, hogy az ügyfél előző három foglalása átlagosan hány főre történt! Az oszlop neve legyen 'Előző három foglalás átlaga'

c. A listában azon vendégek is jelenjenek meg, akik még nem foglaltak!

In [ ]:
SELECT *, v.NEV,
        AVG((f.FELNOTT_SZAM+f.GYERMEK_SZAM))
           OVER(ORDER BY (f.FELNOTT_SZAM+f.GYERMEK_SZAM) ROWS BETWEEN 3 PRECEDING AND 0 FOLLOWING) AS 'Előző három foglalás átlaga'
FROM Vendeg v LEFT JOIN Foglalas f ON f.UGYFEL_FK = v.USERNEV
WHERE (f.FELNOTT_SZAM+f.GYERMEK_SZAM) <
(
 SELECT avg(FELNOTT_SZAM+GYERMEK_SZAM)
 FROM Foglalas
)

Rangsoroljuk az egyes szobákat a foglalások száma, azon belül férőhely szerint!

- Holtverseny esetén a sorszámok sűrűn kövessék egymást
- Az 1-es számot a legtöbbet foglalt, azon belül legkevesebb férőhelyű szoba kapja
- Csak a szoba azonosítója, a férőhely és a helyezés jelenjen meg

In [ ]:
SELECT sz.SZOBA_ID, sz.ferohely,
        RANK()  OVER (PARTITION count(f.FOGLALAS_PK) ORDER BY sz.ferohely DESC)
FROM Szoba sz JOIN Foglalas f ON sz.SZOBA_ID=f.SZOBA_FK